# Indonesia 2000an

Pada notebook ini, saya akan mengeksplorasi lagu-lagu tenar di Indonesia periode 2000-an berdasarkan kata-kata pada liriknya.
Lagu tenar saya definisikan sebagai lagu yang terdapat pada *daftar putar* Spotify [Indonesia 2000an](https://open.spotify.com/playlist/758B9GF7qrfggUiLkhEC1q). Lagu-lagu yang tidak memiliki lirik yang benar pada [genius.com](https://genius.com) akan saya abaikan.

Eksplorasi ini dibuat hanya untuk *lucu-lucuan* saja, tapi kalau teman-teman melihat ada yang bisa diperbaiki, bisa membuat isu pada repositori ini.

Pertama-tama, saya butuh meng-*install* dan mempersiapkan pustaka tambahan yang saya butuhkan untuk mengolah data ini.

In [1]:
import sys
!conda install --yes nltk

Solving environment: done

# All requested packages already installed.



# Pembersihan Kata

Lirik yang saya dapatkan dari Genius masih berupa lirik yang memiliki format tampilan (misal baris baru atau anotasi untuk bagian lagu seperti *verse* atau *chorus*).

Saya akan membersihkan datanya terlebih dahulu. Proses pembersihan yang saya lakukan:
- Membuat semua huruf dalam bentuk huruf kecil untuk mempermudah pemrosesan
- Memisahkan setiap lirik berdasarkan baris baru. Ini perlu saya lakukan karena saya lihat anotasi bagian lagu biasanya ada pada baris sendiri
- Membuang anotasi bagian lagu seperti `[verse]` atau `[chorus]`
- Memisahkan setiap kata dengan spasi
- Membersihkan setiap kata dari tanda baca yang tertinggal pada awal dan akhir kata, misalkan pada kata "aku...", tiga titik terakhir akan dihapus dari kata tersebut. Tanda baca ini digunakan sebagai panduan intonasi pada lirik lagu asli (misal memberi efek sendu, penekanan, atau pemisahan)

In [2]:
# Convert all lyrics to token of words
import json 
LYRIC_FILE = '../raw/lyric/758B9GF7qrfggUiLkhEC1q.json' # Indonesia 2000an

raw_json = {}

with open(LYRIC_FILE, 'r') as f:
    raw_json = json.load(f)
    
raw_json.get("lyrics", [{}])[0]
raw_lyrics = raw_json.get('lyrics', [])
all_lyrics = list(map(lambda x: x.get('lyric', ''), raw_lyrics))

all_lyrics[0]

"Disini kau dan Aku\nTerbiasa Bersama\nMenjalani Kasih Sayang\nBahagia Ku Dengan Mu...\nPernahkah Kau Menguntai\nHari Paling Indah\nKu Ukir Nama Kita Berdua\nDisini Surga Kita\nBila Kita Mencintai Yang Lain\nMungkinkah Hati Ini Akan Tegar\nSebisa Mungkin Tak Akan Pernah\nSayang Ku Akan Hilang\nIf you love somebody\nCould we be this strong\nI will fight to win\nOur love will conquer all\nWouldn't risk my love\nEven just one night\nOur love will stay in my heart\nMy heart\nPernahkah Kau Menguntai\nHari Paling Indah\nKu Ukir Nama Kita Berdua\nDisini Surga Kita\nBila KIta Mencintai Yang Lain\nMungkinkah Hati Ini Akan Tegar\nSebisa Mungkin Tak Akan Pernah\nSayang Ku Akan Hilang..\nIf you love somebody\nCould we be this strong\nI will fight to win\nOur love will conquer all\nWouldn't risk my love\nEven just one night\nOur love will stay in my heart\nMy heart\nPernahkah kau menguntai\nHari paling indah\nKu ukir nama kita berdua\nDisini surga kita\nBila kita mencintai yg lain\nMungkin kah hati

In [3]:
SAMPLE = 5

sanitized_lyrics = []

for lyr in all_lyrics:
    lyr = lyr.lower()
    lyrs = lyr.split('\n') # Split by newline
    lyrs = [l for l in lyrs if len(l) > 0 and l[0] != '[' and l[-1] != ']'] # Avoid [chorus] [verse-1] etc
    sanitized_lyrics.extend(lyrs)
    
sanitized_lyrics[:SAMPLE]

['disini kau dan aku',
 'terbiasa bersama',
 'menjalani kasih sayang',
 'bahagia ku dengan mu...',
 'pernahkah kau menguntai']

In [4]:
words = []

for lyric in sanitized_lyrics:
    words.extend(lyric.split())
    
# Remove trailing dot or comma
def stripnonalpha(w):    
    st, en = -1, -1
    
    for i, c in enumerate(w):
        if c.isalpha():
            if st == -1:
                st = i
            en = i
    
    if st == -1 or en == -1:
        # No nonalpha
        return ''
        
    return w[st:en+1]

words = [stripnonalpha(w) for w in words]

# Filter 'unusable' word
unusable = set(['', 'reff'])

words = [w for w in words if w not in unusable]
    
words[:SAMPLE]

['disini', 'kau', 'dan', 'aku', 'terbiasa']

# Explorasi Data
Pada bagian ini, saya akan mencoba menjawab beberapa hipotesis/pertanyaan saya seputar kata-kata pada lagu tenar.

In [5]:
# Helper function that helps the data processing or data visualization
    
import pandas as pd
def as_table(word_freqs):
    word_list, count_list = [], []
    for w, c in word_freqs:
        word_list.append(w)
        count_list.append(c)

    return pd.DataFrame.from_dict({"words": word_list, "count": count_list})
    
def freq_word_order(w):
    # (word, freq)
    # order by freq DESC, if freq same, sort by word ASC
    return -w[1], w[0]

from collections import defaultdict

## Pemendekan Kata
Pada saat saya membersihkan data, saya menemukan banyak kata yang ditulis dengan pemendakan sebagai panduan pengucapan pada lagu. Contohnya adalah penggunaan kata `s'gala` untuk kata `segala` karena dalam menyanyikan lagu tersebut, huruf `e` tidak gamblang diucapkan.

Saya ingin melihat kata-kata apa saja yang lazim dibuat menjadi bentuk pendeknya (catatan: saya tidak tahu istilah baku untuk bentuk ini).


In [6]:
# Check for contracted words ex: s'mua, s'gala
contracted = defaultdict(int)

for w in words:
    if "'" in w or "`" in w:
        contracted[w] += 1
        
contracted_tp = sorted(contracted.items(), key=freq_word_order)

df = as_table(contracted_tp)

df

,words,count
0,you're,31
1,i'm,26
2,s'lalu,20
3,t'lah,18
4,s'moga,9
5,don't,6
6,s'lamat,6
7,everything's,5
8,it's,5
9,s'lama,5


Hmmm, di luar kata-kata dalam Bahasa Inggris, ada pola menarik yang saya temukan:
- Huruf yang paling sering di-*abai*-kan adalah huruf `e`. Contohnya pada kata `s'gala`, `t'lah`, `kar'na`.
- Huruf awal yang sering mengikuti huruf `e` tersebut adalah huruf `s`.

Ada penjelasan ilmiahnya dalam tata bahasa Indonesia tidak ya? Hipotesis saya sih, mungkin karena huruf `e` adalah salah satu huruf yang punya banyak pengucapan berbeda, misal pada kata `meja` dan `teduh`. Sepertinya sih pemendekan ini lazim terjadi untul lafal `e` pada `teduh`.

# Kata Ulang
Salah satu guna kata ulang adalah untuk menunjukkan nominal `banyak`. Kata ulang apa yang sering digunakan pada lagu tenar di Indonesia?


In [7]:
# Check for repeated words
reps = defaultdict(int)

for w in words:
    if "-" in w:
        reps[w] += 1
        
reps_tp = sorted(reps.items(), key=freq_word_order)

df = as_table(reps_tp)

df[:10]

,words,count
0,dosa-dosaku,10
1,baik-baik,6
2,o-oh,6
3,bintang-bintang,5
4,kadang-kadang,5
5,kata-kata,5
6,siapa-siapa,5
7,ah-ah,4
8,mengiris-iris,4
9,na-na-na-na-na,4


Hmm, sepertinya tidak banyak yang bisa saya simpulkan sih. 
`dosa-dosaku` sering muncul pada lirik lagu `Andai Ku Tahu` dari `Ungu`.

Oh iya, ini membuat saya ingat bahwa data lirik ini sendiri bisa jadi tidak valid karena adanya perbedaan format dalam penulisan lirik. Ada lagu yang menuliskan gamblang lirik yang diulang berkali-kali dan adapula yang menuliskan dengan menggunakan format semaca `(x 2)` atau `back to reff`.

# Kata Dominan
Saya penasaran, kata apa sih yang paling sering muncul di lirik lagu-lagu tenar ini? Hipotesis saya, kata yang akan dominan adalah kata `cinta` karena topik banyak berkutat tentang `cinta`.

In [8]:
wholes = defaultdict(int)

for word in words:
    if word in contracted or word in reps:
        continue
        
    if all([c.isalpha() for c in word]):
        wholes[word] += 1
    
wholes_tp = sorted(wholes.items(), key=freq_word_order)

df = as_table(wholes_tp)
df[:10]

,words,count
0,kau,1128
1,aku,1043
2,ku,959
3,yang,911
4,tak,749
5,dan,479
6,cinta,457
7,di,426
8,ini,299
9,ada,248


Ah, saya benar, kata `cinta` menjadi salah satu kata yang muncul paling sering dari lirik lagu.

Tapi, ternyata kata yang justru paling dominan adalah kata ganti orang seperti `aku`, `ku`, dan `kau`. Hal ini bisa jadi terjadi karena lagu Indonesia sering menjadi lagu yang dinyanyikan kepada orang lain sehingga akan muncul banyak kata ganti orang.

## Sudut Pandang Lagu Dominan

Saya jadi penasaran, kalau kita mengelompokkan kata-kata berdasarkan jenis kata ganti nya, kira-kira pihak mana yang paling dominan diceritakan pada lagu ya?

Kata ganti akan saya kelompokan menjadi tiga (sumber: [dosenbahasa](https://dosenbahasa.com/jenis-jenis-kata-ganti), dengan tambahan):
- **Pertama**: `aku`, `saya`, `daku`, `diriku`, `kita`, `kami`, atau kata yang berakhiran `-ku` (seharusnya saya juga memeriksa apakah kata tersebut **kata kerja transitif**, tapi saya menyederhanakan permasalahan di sini)
- **Kedua**: `kamu`, `anda`, `kau`, `dirimu`, `kalian`, atau kata yang berakhiran `-mu`
- **Ketiga**: `ia`, `dia`, `dirinya`, `beliau`, `mereka`, atau kata yang berakhiran `-nya`

Hipotesis saya, lagu akan dominan dari sudut pandang orang kedua dengan alasan banyak lagu di Indonesia yang dinyanyikan 'seolah-olah' sedang berbicara dengan orang lain (misal: minta maaf, menyatakan cinta, dll).

In [9]:
def is_pertama(w):
    pertama_set = set(['aku', 'ku', 'saya', 'daku', 'diriku', 'kita', 'kami'])
    return (w in pertama_set) or (len(w) > 2 and w[-2:] == 'ku')

def is_kedua(w):
    kedua_set = set(['kamu', 'engkau', 'anda', 'kau', 'dirimu', 'kalian'])
    return (w in kedua_set) or (len(w) > 2 and w[-2:] == 'mu')

def is_ketiga(w):
    ketiga_set = set(['ia', 'dia', 'dirinya', 'beliau', 'mereka'])
    return (w in ketiga_set) or (len(w) > 3 and w[-3:] == 'nya')

sudut_pandangs = defaultdict(int)

for w, c in wholes_tp:
    if is_pertama(w):
        sudut_pandangs["pertama"] += c
    if is_kedua(w):
        sudut_pandangs["kedua"] += c
    if is_ketiga(w):
        sudut_pandangs["ketiga"] += c
        
sudut_pandangs_tp = sorted(sudut_pandangs.items(), key=freq_word_order)

df = as_table(sudut_pandangs_tp)
df

,words,count
0,pertama,3880
1,kedua,2902
2,ketiga,1099


Wah, ternyata hipotesis saya. Ternyata, **sudut pandang orang pertama** lebih dominan untuk digunakan.

## Kata Inti Dominan
Dari kumpulan kata dominan, kita melihat ada banyak kata ganti orang dan juga *stopwords*. *Stopwords* bisa diartikan sebagai kata yang tidak memberikan makna tambahan dari sebuah kalimat, misalkan `yang`, `dan`, dsb. 

Sekarang, saya ingin fokus pada kata-kata inti saja.

In [10]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muzavan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stops = set(stopwords.words('indonesian'))

def is_sudut_pandang(w):
    return is_pertama(w) or is_kedua(w) or is_ketiga(w)

cores = defaultdict(int)
for w, c in wholes_tp:
    if w in stops or is_sudut_pandang(w):
        continue
        
    cores[w] += c
    
cores_tp = sorted(cores.items(), key=freq_word_order)
df = as_table(cores_tp)
df[:20]

,words,count
0,cinta,457
1,hati,221
2,oh,205
3,tuk,156
4,you,128
5,takkan,127
6,pergi,126
7,i,121
8,sayang,90
9,mengerti,77


Yeay, kali ini, hipotesis saya benar! Kata yang berkaitan dengan cinta dan perasaan seperti `cinta`, `hati`, `sayang`, dan `love` memang menjadi kata yang dominan muncul pada lirik lagu Indonesia. 

Beberapa temuan menarik:
- Kata senandung `oh` dan `ah` juga ternyata sering digunakan walaupun tidak memiliki arti tertentu.
- Hidup dan mati memiliki frekuensi yang cukup mirip. Bisa jadi kedua kata ini sering digunakan bersamaan.

Hmm, kira-kita hal apa lagi ya yang bisa kita *ulik* dari data lirik lagu?